In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import utils as ut
from PIL import Image

In [2]:
# make def
def cart2hom( pts_3d):
    """ Input: nx3 points in Cartesian
        Oupput: nx4 points in Homogeneous by pending 1
    """
    n = pts_3d.shape[0]
    pts_3d_hom = np.hstack((pts_3d, np.ones((n, 1))))
    return pts_3d_hom

def project_velo_to_ref( V2C, pts_3d_velo):
    pts_3d_velo = cart2hom(pts_3d_velo)  # nx4
    return np.dot(pts_3d_velo, np.transpose(V2C))

def project_ref_to_rect(R0, pts_3d_ref):
    """ Input and Output are nx3 points """
    return np.transpose(np.dot(R0, np.transpose(pts_3d_ref)))

def project_velo_to_rect( V2C, pts_3d_velo, R0):
        pts_3d_ref = project_velo_to_ref(V2C, pts_3d_velo)
        return project_ref_to_rect(R0,pts_3d_ref)


In [ ]:
# make path
path_image = "sample_data/object/image/front/"
path_lidar = "sample_data/object/roof/"
path_camera = "sample_data/object/front/"
path_calib = "sample_data/object/calib/front/"
path_front_left = "sample_data/object/front_left/"
path_front_right = "sample_data/object/front_right/"
path_roof = "sample_data/object/roof/"
image_path_list = os.listdir(path_image)
lidar_path_list = os.listdir(path_lidar)
camera_path_list = os.listdir(path_camera)
calib_path_list = os.listdir(path_calib)
front_left_path_list = os.listdir(path_front_left)
front_right_path_list = os.listdir(path_front_right)
roof_path_list = os.listdir(path_roof)

In [2]:
import os

path_check_camera = '../../../'
path_check_lidar = '../라벨링데이터/lidar'
path_check_camera2 = '../원천데이터/camera'
path_check_lidar2 = '../원천데이터/camera'

check_camera_list = os.listdir(path_check_camera)
check_lidar_list = os.listdir(path_check_lidar)
check_camera2_list = os.listdir(path_check_camera)
check_lidar2_list = os.listdir(path_check_lidar)


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '../../라벨링데이터/camera'

In [3]:
import json
def make_json_data(path,name):
    with open(f'{path}{name}',"r") as f:
        contents = f.read()
        json_data = json.loads(contents)
    return json_data
def read_calib_Tr_R0(path_calib,image_name):
    with open(f'{path_calib}{image_name.split(".")[0]}.txt','r') as f:
        Tr_velo_to_cam = []
        R0 = []
        contents_calib = f.readlines()
        data_calib = contents_calib[2].split(' ')
        data_R0_s = contents_calib[1].split(' ')
        data_P2 = contents_calib[0].split(' ')
        for calib in data_calib[1:]:
            if calib == data_calib[-1]:
                Tr_velo_to_cam.append(float(calib[:-1]))
            else:
                Tr_velo_to_cam.append(float(calib))
        for R0s in data_R0_s[1:]:
            if R0s == data_R0_s[-1]:
                R0.append(float(R0s[:-1]))
            else:
                R0.append(float(R0s))
        R0 = np.reshape(R0, [3, 3])
    return Tr_velo_to_cam, R0
def aihub_to_kitti_format_camera_cordinate(json_data, image_name,Tr_velo_to_cam, R0, lidar_location):
    # make id check list and txt
    names = image_name.split(".")[0]
    names= names.split("_")
    name = ''
    for i in names:
        name+= i
    # make image
    im = Image.open(f'sample_data/object/image/front/{image_name.split(".")[0]}.jpg').convert('RGB')
    im.save(f'sample_data/object/translated/image_2/{name}.png','png')
    # read velodyne
    with open(f'sample_data/object/velodyne/{image_name.split(".")[0]}.bin','rb') as f:
        contents_velodyne = f.read()
    # make velodyne
    f2 = open(f'sample_data/object/translated/velodyne/{name}.bin','wb')
    f2.write(contents_velodyne)
    # read Carib_Ai_hub
    with open(f'{path_calib}{image_name.split(".")[0]}.txt','r') as f:
        contents_calib = f.readlines()
    # Save Carib_kitti_format
    with open(f'sample_data/object/translated/kitti_format_carib/{lidar_location}/{name}.txt','w') as f:
        p0 = 'p0: 1056.437682 0.0 974.398942 0.0 0.0 1024.415886 583.178996 0.0 0.0 0.0 1.0 0.0'
        p1 = 'p1: 1056.437682 0.0 974.398942 0.0 0.0 1024.415886 583.178996 0.0 0.0 0.0 1.0 0.0'
        p3 = 'p3: 1056.437682 0.0 974.398942 0.0 0.0 1024.415886 583.178996 0.0 0.0 0.0 1.0 0.0'
        tr_imu = 'Tr_imu_to_velo: -0.999069645887872 0.030746732104378723 -0.030240389058074302 0.9276477826710018 -0.030772133023495043 -0.9995263554968372 0.0003748284867846508 0.058829066929946106 -0.03021454111295518 0.0013050410383379344 0.999542584572174 0.21383619101949458'
        f.write(p0 + '\n' + p1 + '\n' + contents_calib[0] + p3 + '\n'  + contents_calib[1] + contents_calib[2] + tr_imu)
    save_to_kitti_str = ''
    for lidar_annotations in json_data['annotations']:
        # read aihub annotation
        category = str.title(str(lidar_annotations['3dbbox.category']))
        # label category change
        label_check_list = ['Truck', 'Bus', 'Car', 'Pedestrian', 'Bicycle']
        if category not in label_check_list:
            continue
        if category == 'Truck' or category == 'Bus':
            category = 'Car'
        if category == 'Bicycle':
            category = 'Cyclist'
        truncated = str(lidar_annotations['3dbbox.truncated'])
        occuluded = str(lidar_annotations['3dbbox.occluded'])
        rotatin_y = str(lidar_annotations['3dbbox.rotation_y'])
        l_3d, h_3d, w_3d = str(lidar_annotations['3dbbox.dimension'][0]), str(lidar_annotations['3dbbox.dimension'][1]), str(lidar_annotations['3dbbox.dimension'][2])
        x_3d, y_3d, z_3d = str(lidar_annotations['3dbbox.location'][0]), str(lidar_annotations['3dbbox.location'][1]), str(lidar_annotations['3dbbox.location'][2])
        z_3d_to_kitti = str(lidar_annotations['3dbbox.location'][2] - lidar_annotations['3dbbox.dimension'][1]/2)
        rotatin_y = -(float(rotatin_y) + 1.5707963268)
        while rotatin_y < -np.pi or rotatin_y > np.pi:
            if rotatin_y < -np.pi:
                rotatin_y += 2*np.pi
            elif rotatin_y > np.pi:
                rotatin_y -= 2*np.pi
        rotatin_y = str(rotatin_y)
        # translate lidar to camera
        v_to_c= np.array(Tr_velo_to_cam)
        V2C = np.reshape(v_to_c, [3, 4])
        xyz_3d = np.array([[float(x_3d), float(y_3d), float(z_3d_to_kitti)]])
        project_xyz_3d = project_velo_to_rect(V2C, xyz_3d, R0)
        x_p,y_p,z_p = project_xyz_3d[0][0], project_xyz_3d[0][1], project_xyz_3d[0][2]
        # check object position
        if float(z_p) < 0 :
            continue
        p2_list = []
        p2 = contents_calib[0][:-1]
        p2 = p2.split(' ')[1:]
        for i in p2:
            p2_list.append(float(i))
        p2 = np.array(p2_list).reshape([3,4])
        x_min, y_min, x_max, y_max = ut.return_bboxes(float(x_p), float(y_p), float(z_p), float(h_3d), float(w_3d), float(l_3d), float(rotatin_y), p2)
        save_to_kitti_str += f"{category} {truncated} {occuluded} {rotatin_y} {x_min} {y_min} {x_max} {y_max} {h_3d} {w_3d} {l_3d} {str(x_p)} {str(y_p)} {str(z_p)} {rotatin_y}\n"
    save_to_kitti_str = save_to_kitti_str[:-1]
    if save_to_kitti_str == '':
        return
    # Save Lidar_to_cam
    with open(f'sample_data/object/translated/lidar_to_cam/{lidar_location}/{name}.txt', 'w') as f:
        f.write(save_to_kitti_str)
    # make train.txt
    f2 = open(f'sample_data/object/translated/ImageSets/train.txt','a')
    f2.write(name + '\n')

In [ ]:
import json

# Aihub to kitti
for image_name in camera_path_list:
    # Make json data
    json_data_camera = make_json_data(path_camera,image_name)
    json_data_fl = make_json_data(path_front_left,image_name)
    json_data_fr = make_json_data(path_front_right,image_name)
    json_data_roof = make_json_data(path_roof,image_name)

    # read Tr_velo_to_cam , R0 from calib
    Tr_velo_to_cam, R0 = read_calib_Tr_R0(path_calib,image_name)
    
    # make roof lidar label
    aihub_to_kitti_format_camera_cordinate(json_data_roof, image_name,Tr_velo_to_cam, R0, 'roof')

In [17]:
# cam_to_image
def custom_get_label_objects(label_dir, idx):
    label_filename = os.path.join(label_dir, f"{idx}")
    return ut.read_label(label_filename)
def get_calibration(calib_path, idx):
    calib_filename = os.path.join(calib_path, f"{idx}")
    return ut.Calibration(calib_filename)
# lidar_location = 'roof'
path_label= 'D:/naver/Project/final-project-level3-cv-18/mmdetection3d/data/kitti/training/label_2'
label_list = os.listdir(path_label)



check_list_false = []

for target in label_list:
    CONFIG = {
    "calib_path" : 'D:/naver/Project/final-project-level3-cv-18/mmdetection3d/data/kitti/training/calib',
    "label_path" : 'D:/naver/Project/final-project-level3-cv-18/mmdetection3d/data/kitti/training/label_2',
    "image_path" : 'D:/naver/Project/final-project-level3-cv-18/mmdetection3d/data/kitti/training/image_2/',
    "target_idx" : target,
}

    target_objects = custom_get_label_objects(CONFIG['label_path'], CONFIG['target_idx'])
    calib = get_calibration(CONFIG['calib_path'], CONFIG['target_idx'])
    # for obj in target_objects:
    #     obj.print_object()
    print( os.path.join(CONFIG['image_path'], f"{CONFIG['target_idx'][:-4]}.jpg"))
    target_image = cv2.cvtColor(cv2.imread(os.path.join(CONFIG['image_path'], f"{CONFIG['target_idx'][:-4]}.png")), cv2.COLOR_BGR2RGB)
    img = ut.show_image_with_boxes(target_image, target_objects, calib)
    cv2.imwrite(f'D:/naver/Project/final-project-level3-cv-18/KITTIVisualizer/save_image/{target}.png',img)

    plt.imshow(img)

   


D:/naver/Project/final-project-level3-cv-18/mmdetection3d/data/kitti/training/image_2/1112021090601800000000.jpg
[[1.05643768e+03 0.00000000e+00 9.74398942e+02 0.00000000e+00]
 [0.00000000e+00 1.02441589e+03 5.83178996e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
KITTIObject
 - Difficulty:Unknown
 - Label:Car | Truncation:0.0 | Occlusion:1 | Alpha:-1.5792354960653299
 - 2DBbox @ Pos:(1920.0,611.5872802734375),(1920.0,1200.0)
 - 3DBbox @ Pos:(7.2793110831763626,1.9053044349924435,4.212226463959539) | Yaw:-1.5792354960653299 | HWL:(1.6680264472961426,2.053748369216919,4.4956464767456055)
KITTIObject
 - Difficulty:Easy
 - Label:Car | Truncation:0.0 | Occlusion:0 | Alpha:-1.5431647981244438
 - 2DBbox @ Pos:(1091.7530517578125,610.0531616210938),(1203.7584228515625,691.3600463867188)
 - 3DBbox @ Pos:(3.853860869440108,2.3198765575547813,24.208538642656553) | Yaw:-1.5431647981244438 | HWL:(1.6260199546813965,1.951796054840088,4.428915977478027)
KITTIObj

KeyboardInterrupt: 

SyntaxError: unexpected EOF while parsing (<ipython-input-2-0934341d3a37>, line 3)